# Исследование уровня потребительской лояльности

## Цель и задачи исследования

**Заказчик исследования** — телекоммуникационная компания (NDA), оказывающая услуги на территории всего СНГ.

**Цель исследования** — определить текущий уровень потребительской лояльности в компании на основе показателя NPS.

**Задачи исследования**:
- подключиться к базе данных,
- выгрузить и провести предобработку данных при помощи SQL, а также получить итоговый датасет в формате csv,
- создать дашборд в Tableau на базе полученного датасета, в котором отобразить распределение участников по полу и возрасту, количеству дней «жизни» клиента, географии, лояльности, общему NPS, а также более подробное описание категории «cторонников».

**Источник данных** — данные предоставлены локально в SQLite в формате нескольких таблиц: `user` - информация о пользователях, `location` - справочник территорий, `age_segment` - информация о возрастных сегментах клиентах, `traffic_segment` - информация по потребляемому трафику, `lifetime_segment` - информация о выделяемых сегментах по количеству месяцев «жизни» клиента. Таблицы соединяются между собой посредством ключей, указанных в прилагаемой к исследованию документации.


## Подключение к базе данных

Для получения датасета необходимо подключиться к базе данных, представленной в формате `.db`. Для этого импортируем библиотеки `os`, отвечающей за системные настройки, `pandas` для выгрузки полученного датасета, `numpy` для работы с числовыми значениями, а также модуль `create_engine` библиотеки `sqlalchemy` для работы с SQL-запросами в рамках jupyter-тетрадки и pandas.

In [1]:
# импортируем необходимые библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

Далее пропишем путь к базе данных в зависимости от пути расположения файла (на практике можно было бы оставит один путь, но так как мы работаем локально или на платформе, то лучше прописать оба).

In [2]:
# объявим переменные с возможными путями, а также основную переменную path_to_db, куда впоследствии будет сохраняться реализуемый
# путь
path_to_db_local = 'db.db'
path_to_db_platform = '/datasets/db.db'
path_to_db = None

# при помощи библиотеки os определим путь файла и запишем его в переменную path_to_db
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# создадим переменную с базой данных, которую будем использовать впоследствии с pandas
if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Составление запроса и экспорт датасета

Далее сформируем запрос: в рамках исследования необходимо объединить 4 таблицы, при этом срезы данных делаться не планируются, поэтому для удобства разбиения запросы на подзапросы воспользуемся табличными выражениями с `WITH`. При этом сами таблицы будем объединять при помощи `LEFT JOIN`: в случае того, если количество данных в справочниках по каким-то причинам отличаются от основной таблицы `user`, то в итоговый датасет попадут только те строки, которые присутствуют в таблице `user`.

**В исследовании были сделаны следующие подзапросы:**
- `new` - дополнительный столбец с категорией новизны клиенты;
- `gs` - дополнительный столбец с изменением типа переменной с числовой на строковую в столбце с полом;

In [3]:
# осуществляем запрос и сохраняем его в переменную query
query = """
        WITH 
        new AS (SELECT user_id, 
                            CASE
                             WHEN lt_day <= 365 THEN 'новые'
                             WHEN lt_day > 365 THEN 'старые'
                            END AS is_new
                    FROM user),
        gs AS (SELECT user_id,
                               CASE
                                WHEN gender_segment = 0 THEN 'мужчина'
                                WHEN gender_segment = 1 THEN 'женщина'
                                ELSE 'другой'
                               END AS gender_segment
                        FROM user)
                        
        SELECT u.user_id,
               u.lt_day,
               n.is_new,
               u.age,
               g.gender_segment,
               u.os_name,
               u.cpe_type_name,
               l.country,
               l.city,
               ags.title AS age_segment,
               trs.title AS traffic_segment,
               lts.title AS lifetime_segment,
               u.nps_score,
               CASE
                   WHEN u.nps_score >= 9 THEN "сторонники"
                   WHEN u.nps_score >= 7 AND u.nps_score <= 8 THEN "нейтралы"
                   WHEN u.nps_score <= 6 THEN "критики"
               END AS nps_group
        FROM user AS u
        LEFT JOIN new AS n ON u.user_id = n.user_id
        LEFT JOIN gs AS g ON u.user_id = g.user_id
        LEFT JOIN location AS l ON u.location_id = l.location_id
        LEFT JOIN age_segment AS ags ON u.age_gr_id = ags.age_gr_id
        LEFT JOIN traffic_segment AS trs ON u.tr_gr_id = trs.tr_gr_id
        LEFT JOIN lifetime_segment AS lts ON u.lt_gr_id = lts.lt_gr_id;
"""

На базе полученного запроса создадим датасет для дальнейшего экспорта в формате `csv` и работе в Tableau. Перед экспортом проверим датасет на корректность составленного запроса и отобразим 5 первых элементов.

In [4]:
# сохраним результаты запроса в переменную df для создания датасета
df = pd.read_sql(query, engine)

# проверим корректность составленного запроса и выведем первые 5 строк датасета на экран
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старые,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,старые,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,старые,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники
3,A004TB,4190,старые,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонники
4,A004XT,1163,старые,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонники


Запрос был составлен корректно, о чем свидетельствует полученный выше датасет. Тем не менее, необходимо провести первичную предобработку данных и скорректировать запрос, в частности:
- отформатировать значения в столбце `city` (и, возможно, в `country`) в силу того, что названия городов, написанных через пробел или дефис, были написаны некорректно;
- убрать нумерацию в значениях категорий в столбцах `age_segment`, `traffic_segment` и `lifetime_segment`. Мне кажется, что нумерация является избыточной: ее можно было бы оставить, если бы, например, в документации к исследованию было сказано, что это кодовые значения, которые необходимо использовать в рамках анализа вместо наименований, однако этого указано не было, поэтому для наглядности оставим именно названия категорий.

Для форматирования значений выведем уникальные значения столбцов `country` и `unique` (про условия использования SQL помню, но Python я использую не столько для предобработки сколько для проведения анализа, чтобы не переписывать запрос несколько раз). 

In [5]:
# выведем уникальные значения стран
df['country'].unique()

array(['Россия'], dtype=object)

In [6]:
# выведем уникальные значения городов
df['city'].unique()

array(['Уфа', 'Киров', 'Москва', 'РостовнаДону', 'Рязань', 'Омск',
       'СанктПетербург', 'Волгоград', 'Тольятти', 'Казань', 'Самара',
       'Красноярск', 'Екатеринбург', 'Калуга', 'Краснодар', 'Иркутск',
       'Пермь', 'Владимир', 'Ижевск', 'Тюмень', 'Оренбург',
       'НижнийНовгород', 'Брянск', 'Челябинск', 'Астрахань', 'Сургут',
       'Тверь', 'Новосибирск', 'НабережныеЧелны', 'Махачкала', 'Воронеж',
       'Курск', 'Владивосток', 'Балашиха', 'Пенза', 'Калининград', 'Тула',
       'Саратов', 'Кемерово', 'Белгород', 'Барнаул', 'Чебоксары',
       'Архангельск', 'Томск', 'Ярославль', 'Ульяновск', 'Хабаровск',
       'Грозный', 'Ставрополь', 'Липецк', 'Новокузнецк', 'Якутск',
       'УланУдэ', 'Сочи', 'Иваново', 'НижнийТагил', 'Смоленск',
       'Волжский', 'Магнитогорск', 'Чита', 'Череповец', 'Саранск'],
      dtype=object)

Видим, что в датасете присутствует только одна страна - Россия, а вот среди городов встречаются некорретные значения, в частности - `РостовнаДону`, `СанктПетербург`, `НижнийНовгород`, `НабережныеЧелны`, `УланУдэ` и `НижнийТагил`. Переименуем данные значения вручную: для автоматизации можно было бы использовать регулярные выражения, но в данном случае количество значений не такое большое, поэтому можно обойтись без них. Также уберем нумерацию из столбцов, обозначенных выше.

In [7]:
# перезаписываем запрос
query = """
        WITH 
        new AS (SELECT user_id, 
                            CASE
                             WHEN lt_day <= 365 THEN 'новые'
                             WHEN lt_day > 365 THEN 'старые'
                            END AS is_new
                    FROM user),
        gs AS (SELECT user_id,
                               CASE
                                WHEN gender_segment = 0 THEN 'мужчина'
                                WHEN gender_segment = 1 THEN 'женщина'
                                ELSE 'другой'
                               END AS gender_segment
                        FROM user)
                        
        SELECT u.user_id,
               u.lt_day,
               n.is_new,
               u.age,
               g.gender_segment,
               u.os_name,
               u.cpe_type_name,
               l.country,
               CASE
                   WHEN l.city = 'РостовнаДону' THEN 'Ростов-на-Дону'
                   WHEN l.city = 'СанктПетербург' THEN 'Санкт-Петербург'
                   WHEN l.city = 'НижнийНовгород' THEN 'Нижний Новгород'
                   WHEN l.city = 'НабережныеЧелны' THEN 'Набережные Челны'
                   WHEN l.city = 'УланУдэ' THEN 'Улан-Удэ'
                   WHEN l.city = 'НижнийТагил' THEN 'Нижний Тагил'
                   ELSE l.city
                   END AS city,
               SUBSTRING(ags.title, 3) AS age_segment,
               SUBSTRING(trs.title, 3) AS traffic_segment,
               SUBSTRING(lts.title, 3) AS lifetime_segment,
               u.nps_score,
               CASE
                   WHEN u.nps_score >= 9 THEN "сторонники"
                   WHEN u.nps_score >= 7 AND u.nps_score <= 8 THEN "нейтралы"
                   WHEN u.nps_score <= 6 THEN "критики"
               END AS nps_group
        FROM user AS u
        LEFT JOIN new AS n ON u.user_id = n.user_id
        LEFT JOIN gs AS g ON u.user_id = g.user_id
        LEFT JOIN location AS l ON u.location_id = l.location_id
        LEFT JOIN age_segment AS ags ON u.age_gr_id = ags.age_gr_id
        LEFT JOIN traffic_segment AS trs ON u.tr_gr_id = trs.tr_gr_id
        LEFT JOIN lifetime_segment AS lts ON u.lt_gr_id = lts.lt_gr_id;
"""

# перезапишем датасет в переменную и выведем первые пять строк на экране
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старые,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,старые,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,старые,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники
3,A004TB,4190,старые,44.0,женщина,IOS,SMARTPHONE,Россия,Ростов-на-Дону,35-44,0.1-1,36+,10,сторонники
4,A004XT,1163,старые,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,сторонники


Видим, что необходимые преобразования в датасете были успешно проведены. Экспортируем датасет в формате `csv` для проведения анализа и проектирования дашборда в Tableau.

In [8]:
# экспортируем датасет в csv-файл
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Проектирование дашборда и презентации в Tableau

В рамках работы над визуализациями, дашбордами и презентацией в Tableau были переименованы поля (для удобства работы), а также рассчитана мера NPS, на базе которой были созданы визуализации, дашборды (общий и 2 детальных, с фильтрами и actions), а также презентация. 

**Ссылка на презентацию Tableau** - https://public.tableau.com/app/profile/pavel.6518/viz/NPS__/NPS_1?publish=yes

## Выводы

**В ходе проведения исследования были получены следующие выводы:**
- В большей степени пользователи представлены в возрасте 25-54 лет, что необходимо учитывать при анализе итогового значения NPS. Так, например, менее лояльными или нейтральными являются более молодые (25-44 года), а более лояльными - старшие пользователи (34-54 года). Вполне вероятно, что если компания сможет увеличить количество лояльных пользователей среди молодежи, то это увеличит NPS.
- По поволому распределению между пользователями нет особых различий: доминирует женский пол, однако разница между полами не очень существенная. При этом наибольшая разница наблюдается при анализе сторонников компании: женщин оказывается все еще не существенно, но больше, чем при других значениях.
- Более 90% пользователей используют смартфоны, менее 2% - планшеты, а также 0.5% - телефоны. 1. При анализе операционных систем выяснили, что в большей степени респонденты используют Android - более 80%, около 12% - iOS, в меньшей степени прочие операционные системы. По количеству трафика большинство пользователей потребляют в районе не менее 1-15 МБ в установленный период. Значение показателя NPS держится на достаточном среднем уровне в районе 20% среди пользователей наиболее популярных устройств, ОС и количества потребляемого трафика. Стоит обратиться внимание на iOS, так как это вторая по популярности среди пользователей ОС, где NPS стремится к нулю.
- Если анализировать соотношение новых и старых пользователей, то можно заметить, что старые пользователи значительно превалируют над новыми. При этом если сравнить их по категориям NPS, то можно заметить, что соотношения между ними не сильно меняются: единственный момент - наибольшая доля новых пользователей приходится в категории сторонников компании, а наименьшая - критиков. Вполне вероятно, что новые пользователи в большей степени склонны к позитивной оценке сервиса, чем к негативной.
- ТОП-10 городов среди пользователей составляют Москва, Санкт-Петербург, Нижний Новгород, Омск, Екатеринбург, Новосибирск, Ростов-на-Дону, Самара и Челябинск. Во всех сегментах NPS лидируют Москва и Санкт-Петербург, а в меньшей степени - Омск, Челябинск и Ростов-на-Дону.